In [1]:
import glob, json, os, sys

sys.path.append('/path/to/metrics/') # TODO: Replace with path to `SWE-bench/metrics` folder
from conversion import convert_log_to_ground_truth
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

Declare repository; Fetch tasks, logs

In [2]:
repo = 'keras_team/keras' # TODO: Replace with repository name
log_dir = '/home/manhtd/Projects/SWE-Python-AI/extracted-data/SWE-Python-AI_validation' # TODO: Replace with path to folder of execution logs

In [3]:
import json

def load_jsonl(file_path):
    """
    Loads a .jsonl file and returns a list of JSON objects.

    Parameters:
        file_path (str): The path to the .jsonl file.

    Returns:
        list: A list of dictionaries or lists, each representing a JSON object.
    """
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data


Get map of version to setup commit

In [4]:
tasks = load_jsonl('/home/manhtd/Projects/SWE-Python-AI/outputs/keras-task-instances-with-version.jsonl') # TODO: Replace with path to versioned candidate task instances
print(len(tasks))
tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)
version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

117


#### Monitor Validation

In [5]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

Total Attempts: 116
Failed: 7
Usable: 109
Corrupt Test: 2
Corrupt Diff: 1
Test Script Timeout: 0
Success E2E: 106


In [6]:
logs_same, logs_diff = monitor_logs_same_diff(log_dir)
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

Logs same: 58
Logs diff: 48


#### Get [FP]2[FP] Tests

In [7]:
inst_id_to_gt = {}
for d in logs_diff:
    status_gt = convert_log_to_ground_truth(d[0])
    inst_id_to_gt[d[0]] = status_gt

#### Create Task Instances `.json` file

In [8]:
import json

def dump_jsonl(data, file_path):
    """
    Dumps a list of JSON objects to a .jsonl file.

    Parameters:
        data (list): A list of dictionaries or lists to write to the file, where each item will be a line.
        file_path (str): The path to the .jsonl file to write to.

    Returns:
        None
    """
    with open(file_path, 'w') as file:
        for item in data:
            # Convert each item to a JSON string and write it as a new line
            file.write(json.dumps(item) + '\n')


In [9]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    if len(v['FAIL_TO_PASS']) == 0:
        continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit[task['version']]
    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))

SAVE_PATH = "../output.jsonl" # TODO: Replace this with a path to a .json file to save the task instances
dump_jsonl(tasks_final, SAVE_PATH)

Final number of task instances:  48


In [10]:
tasks_final[0]

{'repo': 'keras-team/keras',
 'pull_number': 19190,
 'instance_id': 'keras-team__keras-19190',
 'issue_numbers': ['19180'],
 'base_commit': '436937dea3d52eecff3cb6f1bd5161f23c825fae',
 'patch': 'diff --git a/keras/layers/preprocessing/text_vectorization.py b/keras/layers/preprocessing/text_vectorization.py\nindex 2e1fa4633a3..c40715f6c4a 100644\n--- a/keras/layers/preprocessing/text_vectorization.py\n+++ b/keras/layers/preprocessing/text_vectorization.py\n@@ -492,6 +492,10 @@ def from_config(cls, config):\n             config["split"] = serialization_lib.deserialize_keras_object(\n                 config["split"]\n             )\n+\n+        if isinstance(config["ngrams"], list):\n+            config["ngrams"] = tuple(config["ngrams"])\n+\n         return cls(**config)\n \n     def set_vocabulary(self, vocabulary, idf_weights=None):\n',
 'test_patch': 'diff --git a/keras/layers/preprocessing/text_vectorization_test.py b/keras/layers/preprocessing/text_vectorization_test.py\nindex ac3e9